In [1]:
import pandas as pd
import numpy as np

In [9]:
pyf = pd.read_csv("data/clean_ninds_scopus_idx.csv")
pyf.head()

/Users/vitale/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,grant_application_id,grant_type,fiscal_year,support_year,project_start_date,project_end_date,other_project_leader,ninds_aff_name,ninds_aff_type,ninds_aff_city,ninds_aff_state,ninds_aff_country,fiscal_year_total_cost,scopus_idx
0,0,2261377,f32,1993,1,1-apr-1994,NaN,not applicable,yale university,schools of arts and sciences,new haven,ct,united states,21600.0,sandra m aamodt
1,1,2259227,k08,1993,5,1-aug-1988,30-nov-1994,not applicable,stanford university,schools of medicine,stanford,ca,united states,79933.0,ralph d aarons
2,2,3409968,r01,1993,7,1-dec-1986,30-nov-1994,not applicable,university of texas austin,schools of pharmacy,austin,tx,united states,242061.0,creed w abell
3,3,3399173,r01,1993,12,1-may-1982,31-aug-1997,not applicable,emory university,schools of medicine,atlanta,ga,united states,174633.0,ronald f abercrombie
4,4,2262781,r01,1993,14,1-jun-1979,31-dec-1995,not applicable,university of pittsburgh at pittsburgh,schools of medicine,pittsburgh,pa,united states,885354.0,norman s abramson


In [10]:
pyf_means = pyf.groupby("fiscal_year")["fiscal_year_total_cost"].mean().reset_index()
pyf_means.columns = ["fiscal_year","funding"]
pyf_means.set_index("fiscal_year")
pyf_means["funding"] = pyf_means["funding"].round(2)
pyf_means.to_csv("data/per_year_means.csv")

In [11]:
pyf_totals = pyf.groupby("fiscal_year")["fiscal_year_total_cost"].sum().reset_index()
pyf_totals.columns = ["fiscal_year","funding"]
pyf_totals.set_index("fiscal_year")
pyf_totals["funding"] = pyf_totals["funding"].round(2)
pyf_totals.to_csv("data/per_year_totals.csv")

In [12]:
pyf["date"] = pd.to_datetime(pyf['project_start_date'], errors='coerce')

In [13]:
cont_dates = pyf[["fiscal_year_total_cost", "date"]].dropna()
cont_dates = cont_dates.groupby("date")["fiscal_year_total_cost"].sum().reset_index()

In [14]:
cont_dates.to_csv("data/cont_dates.csv")

In [15]:
pyf.head()

,Unnamed: 0,grant_application_id,grant_type,fiscal_year,support_year,project_start_date,project_end_date,other_project_leader,ninds_aff_name,ninds_aff_type,ninds_aff_city,ninds_aff_state,ninds_aff_country,fiscal_year_total_cost,scopus_idx,date
0,0,2261377,f32,1993,1,1-apr-1994,NaN,not applicable,yale university,schools of arts and sciences,new haven,ct,united states,21600.0,sandra m aamodt,1994-04-01
1,1,2259227,k08,1993,5,1-aug-1988,30-nov-1994,not applicable,stanford university,schools of medicine,stanford,ca,united states,79933.0,ralph d aarons,1988-08-01
2,2,3409968,r01,1993,7,1-dec-1986,30-nov-1994,not applicable,university of texas austin,schools of pharmacy,austin,tx,united states,242061.0,creed w abell,1986-12-01
3,3,3399173,r01,1993,12,1-may-1982,31-aug-1997,not applicable,emory university,schools of medicine,atlanta,ga,united states,174633.0,ronald f abercrombie,1982-05-01
4,4,2262781,r01,1993,14,1-jun-1979,31-dec-1995,not applicable,university of pittsburgh at pittsburgh,schools of medicine,pittsburgh,pa,united states,885354.0,norman s abramson,1979-06-01


In [16]:
copy1=pyf.copy()
copy2=pyf.copy()
copy1['project_start_date'] = pd.to_datetime(copy1['project_start_date'], errors='coerce')
copy2['project_end_date'] = pd.to_datetime(copy2['project_end_date'], errors='coerce')
copy1.index=copy1['project_start_date']
copy2.index=copy2['project_end_date']
total_funding_out = copy1.groupby(pd.TimeGrouper('d')).sum()['fiscal_year_total_cost'].fillna(0).subtract(copy2.groupby(pd.TimeGrouper('d')).sum()['fiscal_year_total_cost'].fillna(0),fill_value=0).cumsum()

total_funding_out = total_funding_out.reset_index()
total_funding_out.columns = [["date","fiscal_year_total_cost"]]
total_funding_out.to_csv("data/cont_dates.csv")


/Users/vitale/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  import sys


In [17]:
pyf["ninds_aff_type"] = pyf["ninds_aff_type"].str.replace("/","-")

In [18]:
year_list = pyf["fiscal_year"].unique().tolist()

In [19]:
# total Funding per research organization per year
aff_yearly_total = pyf.groupby(["ninds_aff_name","fiscal_year"])["fiscal_year_total_cost"].sum()
aff_yearly_total = aff_yearly_total.reset_index()
for year in year_list:
    outname = "data/per_aff_fund/" + str(year) + "_aff_year_totals.csv"
    per_year = aff_yearly_total[aff_yearly_total["fiscal_year"]==year].sort_values(ascending=False, by="fiscal_year_total_cost").iloc[0:25]
    fin = per_year[["ninds_aff_name","fiscal_year_total_cost"]]
    fin.to_csv(outname)


In [20]:
# mean funding per research organization per year
aff_yearly_mean = pyf.groupby(["ninds_aff_name","fiscal_year"])["fiscal_year_total_cost"].mean()
aff_yearly_mean = aff_yearly_mean.reset_index()
for year in year_list:
    outname = "data/per_aff_fund/" + str(year) + "_aff_year_means.csv"
    per_year = aff_yearly_mean[aff_yearly_mean["fiscal_year"]==year].sort_values(ascending=False, by="fiscal_year_total_cost").iloc[0:25]
    fin = per_year[["ninds_aff_name","fiscal_year_total_cost"]]
    fin.to_csv(outname)
    
    

In [21]:
pyf_aff = pyf[["ninds_aff_name","fiscal_year","fiscal_year_total_cost"]]
top_25 = pyf_aff.groupby("ninds_aff_name")["fiscal_year_total_cost"].sum().sort_values(ascending=False).iloc[0:25].reset_index()
top_25_list = list(top_25["ninds_aff_name"].unique())
top_affs = pyf_aff[pyf_aff["ninds_aff_name"].isin(top_25_list)].groupby(["ninds_aff_name","fiscal_year"])["fiscal_year_total_cost"].sum().reset_index()
top_affs = top_affs[top_affs["fiscal_year"] != 2019].fillna(0)
# final_top_affs = top_affs.pivot_table(values = "fiscal_year_total_cost",index="fiscal_year",columns ="ninds_aff_name").fillna(0)
top_affs.to_csv("data/top_affs.csv")


In [38]:
# total Funding per type of research organization per year
aff_type_yearly_total = pyf.groupby(["ninds_aff_type","fiscal_year"])["fiscal_year_total_cost"].sum().round(2)
aff_type_yearly_total = aff_type_yearly_total.reset_index()
for year in year_list:
    outname = "data/per_aff_type_fund/" + str(year) + "_aff_type_year_totals.csv"
    per_year = aff_type_yearly_total[aff_type_yearly_total["fiscal_year"]==year].sort_values(ascending=False, by="fiscal_year_total_cost")
    fin = per_year[["ninds_aff_type","fiscal_year_total_cost"]]
    fin.to_csv(outname)

In [39]:
# mean funding per type of research organization per year
aff_type_yearly_mean = pyf.groupby(["ninds_aff_type","fiscal_year"])["fiscal_year_total_cost"].mean().round(2)
aff_type_yearly_mean = aff_type_yearly_mean.reset_index()
for year in year_list:
    outname = "data/per_aff_type_fund/" + str(year) + "_aff_type_year_means.csv"
    per_year = aff_type_yearly_mean[aff_type_yearly_mean["fiscal_year"]==year].sort_values(ascending=False, by="fiscal_year_total_cost")
    fin = per_year[["ninds_aff_type","fiscal_year_total_cost"]]
    fin.to_csv(outname)

In [24]:
pyf_aff_type = pyf[["ninds_aff_type","fiscal_year","fiscal_year_total_cost"]]
aff_type = pyf_aff_type.groupby("ninds_aff_type")["fiscal_year_total_cost"].sum().sort_values(ascending=False).reset_index()
aff_type_list = list(aff_type["ninds_aff_type"].unique())
top_aff_types = pyf_aff_type[pyf_aff_type["ninds_aff_type"].isin(aff_type_list)].groupby(["ninds_aff_type","fiscal_year"])["fiscal_year_total_cost"].sum().reset_index()
top_aff_types = top_aff_types[top_aff_types["fiscal_year"] != 2019].fillna(0)
top_aff_types.to_csv("data/top_aff_types.csv")

In [43]:
# grant type
# total Funding per type of research organization per year
grant_type_yearly_total = pyf.groupby(["grant_type","fiscal_year"])["fiscal_year_total_cost"].sum().round(2)
grant_type_yearly_total = grant_type_yearly_total.reset_index()
for year in year_list:
    outname = "data/per_grant_type_fund/" + str(year) + "_grant_type_year_totals.csv"
    per_year = grant_type_yearly_total[grant_type_yearly_total["fiscal_year"]==year].sort_values(ascending=False, by="fiscal_year_total_cost")
    fin = per_year[["grant_type","fiscal_year_total_cost"]]
    fin.to_csv(outname)

In [44]:
# mean funding per type of research organization per year
grant_type_yearly_mean = pyf.groupby(["grant_type","fiscal_year"])["fiscal_year_total_cost"].mean().round(2)
grant_type_yearly_mean = grant_type_yearly_mean.reset_index()
for year in year_list:
    outname = "data/per_grant_type_fund/" + str(year) + "_grant_type_year_means.csv"
    per_year = grant_type_yearly_mean[grant_type_yearly_mean["fiscal_year"]==year].sort_values(ascending=False, by="fiscal_year_total_cost")
    fin = per_year[["grant_type","fiscal_year_total_cost"]]
    fin.to_csv(outname)

In [52]:
pyf_grant_type = pyf[["grant_type","fiscal_year","fiscal_year_total_cost"]]
grant_type = pyf_grant_type.groupby("grant_type")["fiscal_year_total_cost"].sum().sort_values(ascending=False).reset_index()
grant_type_list = list(grant_type["grant_type"].unique())
top_grant_types = pyf_grant_type[pyf_grant_type["grant_type"].isin(grant_type_list)].groupby(["grant_type","fiscal_year"])["fiscal_year_total_cost"].sum().reset_index()
top_grant_types = top_grant_types[top_grant_types["fiscal_year"] != 2019].fillna(0)
top_grant_types = top_grant_types.sort_values("fiscal_year",ascending=True)
top_grant_types.to_csv("data/top_grant_types.csv")

In [51]:
top_grant_types.sort_values("fiscal_year",ascending=True)

,grant_type,fiscal_year,fiscal_year_total_cost
786,u01,1993,180643.0
316,n43,1993,99899.0
187,k12,1993,418135.0
641,r43,1993,1159493.0
552,r29,1993,18335729.0
208,k17,1993,127303.0
378,p50,1993,24292818.0
323,n44,1993,1220583.0
935,z01,1993,0.0
164,k08,1993,11924440.0
